In [30]:
data = {
  "client0_attacker_14_normal_step": 0.5175706596299099,
  "client0_attacker_19_normal_step": 0.5429864203393666,
  "client0_attacker_32_normal_step": 0.3857142807290817,
  "_step": 4249,
  "client0_loss": 2.031713839054108,
  "client0_attacker_29_tr2t_avg": 0.5275647235774503,
  "client0_attacker_8_normal_avg": 0.6366766778182428,
  "client0_attacker_16_normal_step": 0.5285769033264401,
  "client0_attacker_24_normal_step": 0.544871789879511,
  "client0_attacker_28_normal_step": 0.4467532417557092,
  "client0_test-ppl": 14.366519927978516,
  "client0_attacker_0_normal_avg": 0.5362440276624566,
  "client0_attacker_32_normal_avg": 0.4366247128481753,
  "client0_attacker_9_normal_step": 0.5223380441441088,
  "client0_attacker_34_normal_avg": 0.31704178424232093,
  "client0_attacker_35_normal_avg": 0.27924916250140014,
  "client0_attacker_30_normal_step": 0.4590469049036024,
  "global_step": 4249,
  "client0_global_round": 0,
  "client0_attacker_5_b2tr_avg": 0.6304907277186129,
  "client0_attacker_3_normal_avg": 0.6183885509769569,
  "client0_attacker_23_normal_step": 0.5818181768206444,
  "client0_attacker_31_normal_step": 0.41103848447329144,
  "client0_self_pt": 0,
  "client0_attacker_11_normal_avg": 0.6377722236693276,
  "client0_attacker_33_normal_avg": 0.39310278683275807,
  "client0_attacker_6_normal_step": 0.5439360879591302,
  "client0_attacker_2_normal_avg": 0.6089481650353057,
  "client0_attacker_4_normal_avg": 0.6238578184133798,
  "client0_attacker_22_normal_step": 0.5893812020286877,
  "client0_attacker_25_normal_step": 0.5504761854822313,
  "client0_attacker_13_normal_step": 0.5263951684539971,
  "client0_attacker_9_normal_avg": 0.6384095213150243,
  "client0_attacker_16_normal_avg": 0.6330719245495596,
  "client0_attacker_24_normal_avg": 0.5943958147410345,
  "client0_attacker_26_normal_avg": 0.5723204183075637,
  "client0_attacker_15_normal_step": 0.5175706596299099,
  "client0_attacker_5_b2tr_step": 0.5145243232532479,
  "client0_attacker_7_normal_avg": 0.6351210251036679,
  "client0_attacker_12_normal_avg": 0.6366890118070682,
  "client0_attacker_13_normal_avg": 0.6347554874791649,
  "client0_attacker_21_normal_avg": 0.620009580633759,
  "client0_attacker_25_normal_avg": 0.5867858649221694,
  "client0_attacker_27_normal_avg": 0.5616850112415854,
  "client0_attacker_3_normal_step": 0.5351071642617553,
  "_wandb.runtime": 1649,
  "client0_local_epoch": 0,
  "client0_attacker_10_normal_avg": 0.6380667878549796,
  "client0_attacker_20_normal_avg": 0.624557655192967,
  "client0_attacker_20_normal_step": 0.5729323258308399,
  "client0_attacker_21_normal_step": 0.5688311638336314,
  "client0_attacker_33_normal_step": 0.33369407870222145,
  "client0_attacker_34_normal_step": 0.30359476626650117,
  "client0_attacker_31_normal_avg": 0.4747830768028701,
  "client0_attacker_10_normal_step": 0.5096798162960074,
  "client0_attacker_12_normal_step": 0.5175706596299099,
  "client0_attacker_26_normal_step": 0.5203007468834715,
  "_runtime": 1651.2562370300293,
  "_timestamp": 1707696679.616939,
  "client0_attacker_1_normal_step": 0.5177280500885659,
  "client0_attacker_28_normal_avg": 0.5459204030441271,
  "client0_attacker_6_normal_avg": 0.6324742085562801,
  "client0_attacker_17_normal_avg": 0.6309008512450668,
  "client0_attacker_18_normal_avg": 0.6301045366130664,
  "client0_attacker_4_normal_step": 0.5473022862129748,
  "client0_attacker_17_normal_step": 0.5469824243357921,
  "client0_attacker_15_normal_avg": 0.6324732529184328,
  "client0_attacker_22_normal_avg": 0.611303527540108,
  "client0_attacker_23_normal_avg": 0.6043506538926331,
  "client0_attacker_7_normal_step": 0.5562817669714759,
  "client0_self": 0.0005098648707545071,
  "client0_local_step": 4249,
  "client0_attacker_8_normal_step": 0.5549242374281007,
  "client0_attacker_29_tr2t_step": 0.454184699192842,
  "client0_attacker_2_normal_step": 0.5012778336386043,
  "client0_attacker_11_normal_step": 0.5135746556334841,
  "client0_attacker_0_normal_step": 0.5027100221139359,
  "client0_attacker_14_normal_avg": 0.6337987627417009,
  "client0_attacker_35_normal_step": 0.2107142807178891,
  "client0_attacker_27_normal_step": 0.5097402547483976,
  "client0_attacker_1_normal_avg": 0.5966522677713505,
  "client0_attacker_19_normal_avg": 0.627233252992626,
  "client0_attacker_30_normal_avg": 0.5104403879249436,
  "client0_attacker_18_normal_step": 0.5510835863330007
}

In [1]:
import os
import sys

sys.path.append(os.path.abspath('../..'))
from sfl.utils.exp import get_model_and_tokenizer

model, tokenizer = get_model_and_tokenizer('llama2')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
model.config

LlamaConfig {
  "_name_or_path": "/root/autodl-tmp/sfl/models/daryl149/llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": false,
    "load_in_8bit": true
  },
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vo

In [15]:
from sfl.config import FLConfig
config = FLConfig(collect_intermediates=True,
                  global_round=4,
                  client_evaluate_freq=500,
                  client_epoch=1,  # 每轮联邦每个Client训2轮
                  split_point_1=4,
                  split_point_2=30,  # [0,1 | 2,3,.... 29| 30, 31]
                  use_lora_at_trunk=True,  # 在trunk部分使用LoRA
                  use_lora_at_top=True,
                  use_lora_at_bottom=True,
                  top_and_bottom_from_scratch='False',  # top和bottom都不采用预训练参数.
                  noise_mode="none",
                  noise_scale=0.0,  # 噪声大小,
                  batch_size=2,
                  dataset_type='train'
                  )

model.config_sfl(config)
model.train()

LLAMA2SplitLMHeadModel(
  (model): LLAMA2SplitModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear8bitLt(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Ide

In [4]:
from sfl.utils.model import get_output
from sfl.utils.exp import get_dlg_attacker


ts = get_output("test", tokenizer, model)
print(ts)
dlg = get_dlg_attacker(model)

nobody


In [5]:
from sfl.config import DRAConfig
from sfl.utils.exp import get_dra_attacker

atk_cfg = DRAConfig(target_model_name='llama2', target_dataset='wikitext', target_sps='4-30')
atk, _ = get_dra_attacker(atk_cfg)

In [6]:
from transformers import AdamW
from tqdm import tqdm_notebook
from sfl.utils.model import calculate_rouge
from sfl.utils.exp import get_dataset_class

client_ids = ['0']
# model = model.convert_to_lora_model()
dlg.to(model.device)
atk.to(model.device)
dataset_cls = get_dataset_class('wikitext')
dataset = dataset_cls(tokenizer=tokenizer, client_ids=client_ids)
test_loader = dataset.get_dataloader_unsliced(1, 'train', shrink_frac=0.2)

# avg_rouge = 0
# avg_rouge_dlg = 0
# step = 0
# opt = AdamW(model.parameters(), lr=1e-5)
# with tqdm_notebook(total=len(test_loader)) as pbar:
#   for batch in test_loader:
#     opt.zero_grad()
#     input_ids = batch['input_ids'].to(model.device)
#     o1 = model(input_ids, batch['input_att_mask'].to(model.device), labels=input_ids)
#     # o2 = dlg(tr2t.fx.to(model.device))
#     # print(o1)
#     o1.loss.backward()
#     b2tr, tr2t, all = model.get_all_inter()
#     opt.step()
#     pred = atk(b2tr.fx.to(model.device))
#     # res = tokenizer.decode(pred.argmax(dim=-1)[0])
#       # print(res)
#     # print(batch['input_text'][0])
#     gt = dlg.fit(tr2t.fx.to(model.device), tr2t.grad.to(model.device), epochs=30, gt_init=pred)
#     avg_rouge += calculate_rouge(tokenizer, pred, batch['input_text'])['rouge-l']['f']
#     avg_rouge_dlg += calculate_rouge(tokenizer, gt, batch['input_text'])['rouge-l']['f']
#     step += 1
#     pbar.set_postfix({'dra_rouge': avg_rouge / step, 'dlg_rouge':avg_rouge_dlg/step})
#     pbar.update()


In [22]:
from sfl.utils.exp import add_sfl_params
import argparse
import torch
from typing import Any
from sfl.utils.model import Intermediate
from sfl.simulator.strategy import BaseSFLStrategy
from sfl.simulator.simulator import SFLSimulator

parser = argparse.ArgumentParser()
add_sfl_params(parser)
args = parser.parse_args({})

args.log_to_wandb = False
args.dlg_epochs = 30
args.dlg_init_with_dra = True

# 定义Client本地学习策略
class QAFLStrategy(BaseSFLStrategy):


    def sample_attacker_triggered(self, global_round, client_id, local_epoch, local_step,
                                  b2tr_inter: Intermediate, tr2t_inter: Intermediate,
                                  all_inter: dict[Any, Intermediate],
                                  batch, logs):
        encoder_inter = all_inter.get('encoder', None)
        with torch.no_grad():
            for type, atk in zip(['b2tr', 'tr2t'], [self.dra1, self.dra2]):
                if atk is None:
                    continue
                atk.to(self.simulator.device)
                inter = b2tr_inter if type == 'b2tr' else tr2t_inter
                if self.llm.type == 'encoder-decoder':
                    attacked = atk(torch.concat([encoder_inter.fx.to(
                        self.simulator.device), inter.fx.to(atk.device)], dim=1))
                else:
                    attacked = atk(inter.fx.to(atk.device))
                rouge_res = calculate_rouge(self.tokenizer, attacked, batch['input_text'])
                self.log_to_sample_result(client_id, f'attacker_{type}', rouge_res['rouge-l']['f'])
                self.log_to_all_result(client_id, f'attacker_{type}', rouge_res['rouge-l']['f'])
                logs[f'attacker_{type}_step'] = rouge_res['rouge-l']['f']
        gt_init = None
        if self.args.dlg_init_with_dra:
            gt_init = attacked
        self.dlg.to(self.simulator.device)
        gt = self.dlg.fit(tr2t_inter.fx.to(self.simulator.device), tr2t_inter.grad.to(self.simulator.device),
                          epochs=self.args.dlg_epochs,
                          adjust=False,
                          beta=self.args.dlg_beta,
                          gt_init=gt_init,
                          gt_reg=self.args.dlg_dra_reg,
                          temp_range=self.args.dlg_temp_range,
                          further_ft=self.args.dlg_further_ft,
                          encoder_inter=None if encoder_inter is None else encoder_inter.fx.to(
                              self.simulator.device)
                          )
        if self.llm.type == 'encoder-decoder':
            # replace the latter half of attacked to gt
            attacked[:, -gt.shape[1]:, :] = gt
            rouge = calculate_rouge(self.tokenizer, attacked, batch['input_text'])
        else:
            rouge = calculate_rouge(self.tokenizer, gt, batch['input_text'])
        self.log_to_sample_result(client_id, 'tag_rouge_lf', rouge['rouge-l']['f'])
        self.log_to_all_result(client_id, 'tag_rouge_lf', rouge['rouge-l']['f'])
        print(self.attack_all_performs)



simulator = SFLSimulator(client_ids=client_ids,
                             strategy=QAFLStrategy(args, model, tokenizer, test_loader, atk, None,dlg),
                             llm=model,
                             tokenizer=tokenizer,
                             dataset=dataset, config=config, args=args)

In [23]:
simulator.simulate()

==================================Global Round 0=================================


/root/miniconda3/envs/sfl/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Client 0 Epoch 0 Step (25, 25) Loss 4.160:  50%|██████████████████████████████████████████████                                              | 25/50 [00:10<00:19,  1.28it/s]

{'0': {'attacker_b2tr': [0.8272024232734682], 'tag_rouge_lf': [0.8850751691844674]}}


Client 0 Epoch 0 Step (25, 25) Loss 4.160:  52%|███████████████████████████████████████████████▊                                            | 26/50 [00:12<00:26,  1.12s/it]

{'0': {'attacker_b2tr': [0.8272024232734682, 0.8380681768187558], 'tag_rouge_lf': [0.8850751691844674, 0.8305422597568983]}}


...

{'0': {'attacker_b2tr': [0.8272024232734682, 0.8380681768187558, 0.8364117597065831, 0.9007445768382571, 0.8967741885559293], 'tag_rouge_lf': [0.8850751691844674, 0.8305422597568983, 0.8763371794261045, 0.9359576918279902, 0.8982404642157533]}}


Client 0 Epoch 0 Step (29, 29) Loss 2.846:  60%|███████████████████████████████████████████████████████▏                                    | 30/50 [00:20<00:37,  1.86s/it]

{'0': {'attacker_b2tr': [0.8272024232734682, 0.8380681768187558, 0.8364117597065831, 0.9007445768382571, 0.8967741885559293, 0.8679947818195736], 'tag_rouge_lf': [0.8850751691844674, 0.8305422597568983, 0.8763371794261045, 0.9359576918279902, 0.8982404642157533, 0.9082718007828153]}}


Client 0 Epoch 0 Step (30, 30) Loss 2.826:  60%|███████████████████████████████████████████████████████▏                                    | 30/50 [00:21<00:14,  1.41it/s]


KeyboardInterrupt: 